# Challenge Exercises for CICE

This exercise was created by David Bailey.

<div class="alert alert-info" style="text-align: center;">

[CESM CICE User's Guide](https://www.cesm.ucar.edu/models/cesm2/sea-ice)

</div>

**Learning Goals:**
- Student will learn what a G compset is, the types of forcing available to run one, and how to run one.
- Student will learn how to make a namelist modification that changes snow albedo and compare 


**Details:**

- This exercise uses the same code base as the rest of the tutorial. 
- You will be using the G compset at the T62_g37 resolution.
- You will run a control simulation and a DETAILS. Each simulation will be run for one year. 
- You will then use 'ncview' [LINK HERE] to see howt he experiment differs from the control simulation.

**What is a G case?**

The G compset has active and coupled ocean and sea-ice components. The G compset requires boundary forcing from the atmosphere. The G compset is forced with atmospheric data that does not change interactively as the ocean and sea-ice evolve in time. The land is also not active during a G compset experiment. 

There are two types of temporal forcing for G compsets:
- Normal Year Forcing (NYF) is 12 months of atmospheric data (like a climatology) that repeats every year. NYF is the default forcing.
- Interannual varying forcing (GIAF) is forcing that varies by year over the time period (X-X). 

There are two datasets that can be used for G compsets:
- JRA55 atmospheric data (REFERENCE)
- Coordinated Ocean-ice Reference Experiments (CORE) version 2 atmospheric data (Large and Yeager 2009).


**Add Large and Yeager reference**


<div class="alert alert-info" style="text-align: center;">

[CESM CICE Discussion Forum](https://bb.cgd.ucar.edu/cesm/forums/cesm-cice.137/)

</div>

<div class="alert alert-info">
<strong>Exercise 1: Run a control case</strong><br><br>
 
Create a case called **g_control** using the compset  ``G``  at ``T62_g37`` resolution. 
    
Set the run length to **1 year**. 

Build and run the model. Since this is a control case, we want to build it "out of the box" without any modifications. 

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
    
**How do I compile?**

You can compile with the command:
```
qcmd -- ./case.build
```

**How do I control the output?**

Use namelist variables: ``nhtfrq``, ``mfilt``, ``fincl``. 

Look at the online documentation for these variables.

**How do I check my solution?**

When your run is completed, go to the archive directory. 

(1) Check that your archive directory contains the files:

- ``h0`` files
```
b1850_high_freq.cam.h0.0001-01.nc
```
- ``h1`` files
```
b1850_high_freq.cam.h1.0001-01-01-00000.nc
b1850_high_freq.cam.h1.0001-02-01-00000.nc
```
- ``h2`` files
```
b1850_high_freq.cam.h2.0001-01-01-00000.nc
…
b1850_high_freq.cam.h2.0001-01-31-00000.nc
b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.

```
ncdump -h b1850_high_freq.cam.h1.0001-01-01-00000.nc
ncdump -h b1850_high_freq.cam.h2.0001-01-01-00000.nc
```

(3) Check the number of timesteps in the ``h1`` and the ``h2`` files.
Look at the sizes of the files. 

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>

ADD ALL THE RELEVANT COMMANDS HERE (BELOW IS OLDER INFO BUT PROVIDES GUIDANCE ON THE STYLE)
    
    
    
Create a new case <font face="Courier" color='purple'><strong>b1850_high_freq </strong></font> with the command:
```
cd /glade/p/cesm/tutorial/cesm2.1_tutorial2022/cime/scripts
./create_newcase --case ~/cases/b1850_high_freq  --compset B1850 --res f19_g17 
```

Case <font face="Courier" color='purple'><strong>setup</strong></font>:
```    
cd ~/cases/b1850_high_freq 
./case.setup
```
  
Edit the file <font face="Courier" color='purple'><strong>user_nl_cam</strong></font> and add the lines:
```
 nhtfrq = 0, -24, -3
 mfilt = 1, 31, 8 
 fincl2 = 'T:I','Q:I','U:I','V:I'
 fincl3 = 'T','Q','U','V'
```
    
Change the <font face="Courier" color='purple'><strong>run length</strong></font>:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

If needed, change <font face="Courier" color='purple'><strong>job queue</strong></font> 
and <font face="Courier" color='purple'><strong>account number</strong></font>. 
For instance:
```  
./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
```

Build and submit:
```
qcmd -- ./case.build
./case.submit
```

When the run is completed, look into the archive directory for: 
<font face="Courier" color='purple'><strong>b1850_high_freq</strong></font>.  
    
(1) Check that your archive directory on cheyenne (The path will be different on otehr machines): 
```
cd /glade/scratch/$user/archive/b1850_high_freq/atm/hist
ls 
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.
Look at the variables attributes. What is the difference between the 2 files ? 

The file with the instantaneous output ``h1`` have no cell_methods attribute while the average output ``h2`` has a attribute: 
```
cell_methods = "time: mean"
```

For instance for the field Q.

In the instantaneous file ``b1850_high_freq.cam.h1.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
```
In the time-averaged file  ``b1850_high_freq.cam.h2.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
                Q:cell_methods = "time: mean" ;
```
(3) Check the number of timesteps in the h1 and the h2 files. 

- ``h1`` contains 31 time samples.  
In the netcdf file,  
```
time = UNLIMITED ; // (31 currently)
```
- ``h2`` contains 8 time samples 
In the netcdf file, 
```
time = UNLIMITED ; // (8 currently)
```
- Check the size of the files
```
du –ks –h /glade/scratch/$user/archive/b1850_high_freq/atm/hist/*
```
```
234M    b1850_high_freq.cam.h0.0001-01.nc

210M    b1850_high_freq.cam.h1.0001-01-01-00000.nc
7.0M    b1850_high_freq.cam.h1.0001-02-01-00000.nc

55M     b1850_high_freq.cam.h2.0001-01-01-00000.nc
55M     b1850_high_freq.cam.h2.0001-01-02-00000.nc
...
55M     b1850_high_freq.cam.h2.0001-01-31-00000.nc
7.0M    b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

</details>
</div>



<div class="alert alert-info">
<strong>Exercise 2: Change snow albedo</strong><br><br>
 
Create a case called **g_snowalbedo** using the compset  ``G``  at ``T62_g37`` resolution. 
    
Set the run length to **1 year**. 

Create a clone of your control case.

Find the r_snw namelist parameter (see CaseDocs/ice_in) and increase this value to +2.00 by adding the change to the user_nl_cice file.

Build and run the model for one year. 

Note this snow parameter specifices the number of standard deviations away from the base optical properties in the shortwave radiative transfer code. 

rsnw_nonmelt = 500 - r_Snw*250 (in microns)
higher r_snw --> lower rsnw_nonmelt,
lower rsnw_nonmelt --> higher albedos


Provide info about how to compare the simulations using ncview/ncdiff, etc.

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
    
**How do I compile?**

You can compile with the command:
```
qcmd -- ./case.build
```

**How do I control the output?**

Use namelist variables: ``nhtfrq``, ``mfilt``, ``fincl``. 

Look at the online documentation for these variables.

**How do I check my solution?**

When your run is completed, go to the archive directory. 

(1) Check that your archive directory contains the files:

- ``h0`` files
```
b1850_high_freq.cam.h0.0001-01.nc
```
- ``h1`` files
```
b1850_high_freq.cam.h1.0001-01-01-00000.nc
b1850_high_freq.cam.h1.0001-02-01-00000.nc
```
- ``h2`` files
```
b1850_high_freq.cam.h2.0001-01-01-00000.nc
…
b1850_high_freq.cam.h2.0001-01-31-00000.nc
b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.

```
ncdump -h b1850_high_freq.cam.h1.0001-01-01-00000.nc
ncdump -h b1850_high_freq.cam.h2.0001-01-01-00000.nc
```

(3) Check the number of timesteps in the ``h1`` and the ``h2`` files.
Look at the sizes of the files. 

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>

ADD ALL THE RELEVANT COMMANDS HERE (BELOW IS OLDER INFO BUT PROVIDES GUIDANCE ON THE STYLE)
    
    
    
Create a new case <font face="Courier" color='purple'><strong>b1850_high_freq </strong></font> with the command:
```
cd /glade/p/cesm/tutorial/cesm2.1_tutorial2022/cime/scripts
./create_newcase --case ~/cases/b1850_high_freq  --compset B1850 --res f19_g17 
```

Case <font face="Courier" color='purple'><strong>setup</strong></font>:
```    
cd ~/cases/b1850_high_freq 
./case.setup
```
  
Edit the file <font face="Courier" color='purple'><strong>user_nl_cam</strong></font> and add the lines:
```
 nhtfrq = 0, -24, -3
 mfilt = 1, 31, 8 
 fincl2 = 'T:I','Q:I','U:I','V:I'
 fincl3 = 'T','Q','U','V'
```
    
Change the <font face="Courier" color='purple'><strong>run length</strong></font>:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

If needed, change <font face="Courier" color='purple'><strong>job queue</strong></font> 
and <font face="Courier" color='purple'><strong>account number</strong></font>. 
For instance:
```  
./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
```

Build and submit:
```
qcmd -- ./case.build
./case.submit
```

When the run is completed, look into the archive directory for: 
<font face="Courier" color='purple'><strong>b1850_high_freq</strong></font>.  
    
(1) Check that your archive directory on cheyenne (The path will be different on otehr machines): 
```
cd /glade/scratch/$user/archive/b1850_high_freq/atm/hist
ls 
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.
Look at the variables attributes. What is the difference between the 2 files ? 

The file with the instantaneous output ``h1`` have no cell_methods attribute while the average output ``h2`` has a attribute: 
```
cell_methods = "time: mean"
```

For instance for the field Q.

In the instantaneous file ``b1850_high_freq.cam.h1.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
```
In the time-averaged file  ``b1850_high_freq.cam.h2.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
                Q:cell_methods = "time: mean" ;
```
(3) Check the number of timesteps in the h1 and the h2 files. 

- ``h1`` contains 31 time samples.  
In the netcdf file,  
```
time = UNLIMITED ; // (31 currently)
```
- ``h2`` contains 8 time samples 
In the netcdf file, 
```
time = UNLIMITED ; // (8 currently)
```
- Check the size of the files
```
du –ks –h /glade/scratch/$user/archive/b1850_high_freq/atm/hist/*
```
```
234M    b1850_high_freq.cam.h0.0001-01.nc

210M    b1850_high_freq.cam.h1.0001-01-01-00000.nc
7.0M    b1850_high_freq.cam.h1.0001-02-01-00000.nc

55M     b1850_high_freq.cam.h2.0001-01-01-00000.nc
55M     b1850_high_freq.cam.h2.0001-01-02-00000.nc
...
55M     b1850_high_freq.cam.h2.0001-01-31-00000.nc
7.0M    b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

</details>
</div>



<div class="alert alert-info">
<strong>Exercise 3: Something with source code</strong><br><br>
 
Create a case called **g_snowalbedo** using the compset  ``G``  at ``T62_g37`` resolution. 
    
Set the run length to **1 year**. 

Create a clone of your control case.

SOMETHING NEW ABOUT SRC MODS> NOTE YOU HAVE TO ADD THIS BEFORE BUILDING!

Build and run the model for one year. 

Provide info about how to compare the simulations using ncview/ncdiff, etc.

</div>


<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
    
**How do I compile?**

You can compile with the command:
```
qcmd -- ./case.build
```

**How do I control the output?**

Use namelist variables: ``nhtfrq``, ``mfilt``, ``fincl``. 

Look at the online documentation for these variables.

**How do I check my solution?**

When your run is completed, go to the archive directory. 

(1) Check that your archive directory contains the files:

- ``h0`` files
```
b1850_high_freq.cam.h0.0001-01.nc
```
- ``h1`` files
```
b1850_high_freq.cam.h1.0001-01-01-00000.nc
b1850_high_freq.cam.h1.0001-02-01-00000.nc
```
- ``h2`` files
```
b1850_high_freq.cam.h2.0001-01-01-00000.nc
…
b1850_high_freq.cam.h2.0001-01-31-00000.nc
b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.

```
ncdump -h b1850_high_freq.cam.h1.0001-01-01-00000.nc
ncdump -h b1850_high_freq.cam.h2.0001-01-01-00000.nc
```

(3) Check the number of timesteps in the ``h1`` and the ``h2`` files.
Look at the sizes of the files. 

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>

ADD ALL THE RELEVANT COMMANDS HERE (BELOW IS OLDER INFO BUT PROVIDES GUIDANCE ON THE STYLE)
    
    
    
Create a new case <font face="Courier" color='purple'><strong>b1850_high_freq </strong></font> with the command:
```
cd /glade/p/cesm/tutorial/cesm2.1_tutorial2022/cime/scripts
./create_newcase --case ~/cases/b1850_high_freq  --compset B1850 --res f19_g17 
```

Case <font face="Courier" color='purple'><strong>setup</strong></font>:
```    
cd ~/cases/b1850_high_freq 
./case.setup
```
  
Edit the file <font face="Courier" color='purple'><strong>user_nl_cam</strong></font> and add the lines:
```
 nhtfrq = 0, -24, -3
 mfilt = 1, 31, 8 
 fincl2 = 'T:I','Q:I','U:I','V:I'
 fincl3 = 'T','Q','U','V'
```
    
Change the <font face="Courier" color='purple'><strong>run length</strong></font>:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

If needed, change <font face="Courier" color='purple'><strong>job queue</strong></font> 
and <font face="Courier" color='purple'><strong>account number</strong></font>. 
For instance:
```  
./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
```

Build and submit:
```
qcmd -- ./case.build
./case.submit
```

When the run is completed, look into the archive directory for: 
<font face="Courier" color='purple'><strong>b1850_high_freq</strong></font>.  
    
(1) Check that your archive directory on cheyenne (The path will be different on otehr machines): 
```
cd /glade/scratch/$user/archive/b1850_high_freq/atm/hist
ls 
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.
Look at the variables attributes. What is the difference between the 2 files ? 

The file with the instantaneous output ``h1`` have no cell_methods attribute while the average output ``h2`` has a attribute: 
```
cell_methods = "time: mean"
```

For instance for the field Q.

In the instantaneous file ``b1850_high_freq.cam.h1.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
```
In the time-averaged file  ``b1850_high_freq.cam.h2.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
                Q:cell_methods = "time: mean" ;
```
(3) Check the number of timesteps in the h1 and the h2 files. 

- ``h1`` contains 31 time samples.  
In the netcdf file,  
```
time = UNLIMITED ; // (31 currently)
```
- ``h2`` contains 8 time samples 
In the netcdf file, 
```
time = UNLIMITED ; // (8 currently)
```
- Check the size of the files
```
du –ks –h /glade/scratch/$user/archive/b1850_high_freq/atm/hist/*
```
```
234M    b1850_high_freq.cam.h0.0001-01.nc

210M    b1850_high_freq.cam.h1.0001-01-01-00000.nc
7.0M    b1850_high_freq.cam.h1.0001-02-01-00000.nc

55M     b1850_high_freq.cam.h2.0001-01-01-00000.nc
55M     b1850_high_freq.cam.h2.0001-01-02-00000.nc
...
55M     b1850_high_freq.cam.h2.0001-01-31-00000.nc
7.0M    b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

</details>
</div>

